# 1 Импорт библиотек

In [12]:
import hashlib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 2 Загрузка данных

In [13]:
doc_views = pd.read_csv('doc_views.csv')

In [14]:
payments = pd.read_excel('case_data.xlsx',sheet_name='Payments')

In [15]:
doc_views.head()

,document_id,user_id,viewed_at,document_title,course_name,university_name,category_id,category_name,premium,rating,pages,word_count,ID,Name,Last Name,Passport Number
0,9738742,8469118,2020-11-30 22:59:56,Sosiale medier - 2. time,Sosiale medier,Handelshøyskolen BI,3,Lecture notes,0,0,1,151,1,Jacob,Edwards,1903155229
1,2042372,13198098,2020-11-30 22:59:52,Bedriften-kortsvarsspørsmål,Bedriften,Handelshøyskolen BI,7,Summaries,1,50,23,9551,2,Felicia,Smith,1466748063
2,3653898,11182794,2020-11-30 22:59:41,Sammendrag-i-markedsføring,Markedsføring-grunnkurs,Norges teknisk-naturvitenskapelige universitet,3,Lecture notes,0,1,107,37572,3,Erin,Parker,4864835140
3,7968789,13004670,2020-11-30 22:59:37,"Dydsetikk og abort, Hursthouse",Examen philosophicum,Universitetet i Oslo,8,Mandatory assignments,1,3,6,929,4,Morgan,Mcguire,1837181476
4,2036442,12525936,2020-11-30 22:59:30,Kortsvar-bedriften - Sammendrag Bedriften,Bedriften,Handelshøyskolen BI,7,Summaries,1,228,21,5567,5,Lisa,Erickson,5567324499


In [16]:
payments.tail(10)

,id,created_at,user_id,amount,brand_code,shopper_locale,currency,event_code,is_trial,trial_days
3408,2495154,2020-11-01 12:57:31,12489750.0,0,paypal,no,NOK,AUTHORISATION,1,30
3409,2495004,2020-11-01 12:34:39,12489456.0,0,paypal,no,NOK,AUTHORISATION,1,30
3410,2494949,2020-11-01 12:18:05,12489224.0,0,paypal,no,NOK,AUTHORISATION,1,30
3411,2494822,2020-11-01 11:27:04,12488536.0,0,visa,no,NOK,AUTHORISATION,1,30
3412,2494811,2020-11-01 11:19:18,12488456.0,11997,visa,no,NOK,AUTHORISATION,0,0
3413,2494657,2020-11-01 10:43:11,8227880.0,35988,visa,no,NOK,AUTHORISATION,0,0
3414,2494297,2020-11-01 08:33:23,12487016.0,0,mc,no,NOK,AUTHORISATION,1,30
3415,2494269,2020-11-01 08:05:53,12486818.0,0,visa,no,NOK,AUTHORISATION,1,30
3416,2494216,2020-11-01 07:37:35,12486620.0,0,mc,no,NOK,AUTHORISATION,1,30
3417,2493338,2020-11-01 01:24:26,11413026.0,0,visa,no,NOK,AUTHORISATION,1,30


# 3 Предобработка данных

## Переименование полей

In [17]:
doc_views = doc_views.rename(columns={'Name':'name','Last Name':'last_name', 'Passport Number':'passport_number', 'ID':'id'})

## Преобразование типов данных

In [18]:
 doc_views.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2241 entries, 0 to 2240
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   document_id      2241 non-null   int64 
 1   user_id          2241 non-null   int64 
 2   viewed_at        2241 non-null   object
 3   document_title   2241 non-null   object
 4   course_name      2241 non-null   object
 5   university_name  2241 non-null   object
 6   category_id      2241 non-null   int64 
 7   category_name    1916 non-null   object
 8   premium          2241 non-null   int64 
 9   rating           2241 non-null   int64 
 10  pages            2241 non-null   int64 
 11  word_count       2241 non-null   int64 
 12  id               2241 non-null   int64 
 13  name             2241 non-null   object
 14  last_name        2241 non-null   object
 15  passport_number  2241 non-null   int64 
dtypes: int64(9), object(7)
memory usage: 280.3+ KB


In [19]:
doc_views['viewed_at'] = pd.to_datetime(doc_views['viewed_at'])

In [20]:
doc_views['passport_number'] = doc_views['passport_number'].astype(str)

In [21]:
payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3418 entries, 0 to 3417
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              3418 non-null   int64         
 1   created_at      3418 non-null   datetime64[ns]
 2   user_id         3382 non-null   float64       
 3   amount          3418 non-null   int64         
 4   brand_code      3418 non-null   object        
 5   shopper_locale  3418 non-null   object        
 6   currency        3418 non-null   object        
 7   event_code      3418 non-null   object        
 8   is_trial        3418 non-null   int64         
 9   trial_days      3418 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(4), object(4)
memory usage: 267.2+ KB


In [22]:
payments['user_id'] = payments['user_id'].astype('int')

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

## Обработка пропуска

In [23]:
payments.query('user_id.isna()')

,id,created_at,user_id,amount,brand_code,shopper_locale,currency,event_code,is_trial,trial_days
0,2663933,2020-11-30 23:58:31,NaN,0,visa,no,NOK,AUTHORISATION,1,30
1,2663930,2020-11-30 23:57:31,NaN,0,paypal,no,NOK,AUTHORISATION,1,30
2,2663858,2020-11-30 23:40:27,NaN,0,visa,no,NOK,AUTHORISATION,1,30
3,2663852,2020-11-30 23:39:02,NaN,0,paypal,no,NOK,AUTHORISATION,1,30
4,2663817,2020-11-30 23:31:13,NaN,0,paypal,no,NOK,AUTHORISATION,1,30
5,2663801,2020-11-30 23:27:48,NaN,0,visa,no,NOK,AUTHORISATION,1,30
6,2663770,2020-11-30 23:18:39,NaN,0,paypal,no,NOK,AUTHORISATION,1,30
7,2663763,2020-11-30 23:16:56,NaN,0,visa,no,NOK,AUTHORISATION,1,30
8,2663762,2020-11-30 23:16:19,NaN,0,visa,en_us,NOK,AUTHORISATION,1,30
9,2663743,2020-11-30 23:08:27,NaN,0,visa,no,NOK,AUTHORISATION,1,30


In [24]:
payments = payments.query("user_id.isna() == False")

In [25]:
doc_views.isna().sum()

document_id          0
user_id              0
viewed_at            0
document_title       0
course_name          0
university_name      0
category_id          0
category_name      325
premium              0
rating               0
pages                0
word_count           0
id                   0
name                 0
last_name            0
passport_number      0
dtype: int64

In [26]:
doc_views.query('category_name.isna()')

,document_id,user_id,viewed_at,document_title,course_name,university_name,category_id,category_name,premium,rating,pages,word_count,id,name,last_name,passport_number
20,2057305,11601338,2020-11-30 22:57:52,Sammendrag Bedriften HIS3410,Bedriften,Handelshøyskolen BI,7,NaN,1,28,43,11852,21,Catherine,Moore,5913467767
21,2695140,13049904,2020-11-30 22:57:50,Eksamen 2017,Personalpsykologi,Universitetet i Bergen,4,NaN,1,11,7,4241,22,Arthur,Smith,2752158462
22,2548470,11588852,2020-11-30 22:57:44,Bedriften,Bedriften,Handelshøyskolen BI,3,NaN,1,22,28,4448,23,Ashley,Turner,6746832890
23,3622849,13205250,2020-11-30 22:57:36,STV1400 – OPA – Samleark 1,Offentlig politikk og administrasjon 1 (innfør...,Universitetet i Oslo,7,NaN,1,7,47,6089,24,Mark,Warren,1886590146
24,6092770,11588852,2020-11-30 22:57:17,Web Page - arbeidskrav 5,Bedriftsøkonomisk analyse,Handelshøyskolen BI,8,NaN,1,5,3,487,25,Edward,Benson,9935724825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,4039156,12379916,2020-11-30 22:13:52,Organisasjonsstruktur,Organisasjonsteori og ledelse,Norges Arktiske Universitet,7,NaN,0,7,5,494,341,Lauren,Washington,6327328228
341,3910049,12820726,2020-11-30 22:13:36,"Eksamen 12 Desember 2018, svar",Etikk og samfunnsansvar,Universitetet i Sørøst-Norge,4,NaN,0,9,6,4785,342,Melissa,Morgan,1111171306
342,9161634,9620122,2020-11-30 22:13:35,"Eksamen 5 Mai 2020, svar",Sosiale medier,Handelshøyskolen BI,4,NaN,1,0,13,4740,343,Tammy,Bennett,7369891115
343,2783840,12972078,2020-11-30 22:13:13,"Eksamen 2015, svar",Velferd og økonomisk politikk,Universitetet i Bergen,4,NaN,1,0,2,442,344,Adam,Hood,7535548383


In [27]:
doc_views.query('category_name.isna() == False')[['category_id', 'category_name']]

,category_id,category_name
0,3,Lecture notes
1,7,Summaries
2,3,Lecture notes
3,8,Mandatory assignments
4,7,Summaries
...,...,...
2236,4,Past exams
2237,6,Practical
2238,4,Past exams
2239,7,Summaries


In [28]:
doc_views.query('category_name.isna() == False')[['category_id', 'category_name']].drop_duplicates()

,category_id,category_name
0,3,Lecture notes
1,7,Summaries
3,8,Mandatory assignments
6,4,Past exams
13,10,Essays
351,5,Other
366,9,Tutorial work
370,6,Practical
1439,1,Book solutions


In [29]:
doc_views.query('category_name.isna() == False')[['category_id', 'category_name']].drop_duplicates().reset_index(drop=True)

,category_id,category_name
0,3,Lecture notes
1,7,Summaries
2,8,Mandatory assignments
3,4,Past exams
4,10,Essays
5,5,Other
6,9,Tutorial work
7,6,Practical
8,1,Book solutions


In [30]:
doc_views.query('category_name.isna() == False')[['category_id', 'category_name']].drop_duplicates().reset_index(drop=True).set_index('category_id')['category_name']

category_id
3             Lecture notes
7                 Summaries
8     Mandatory assignments
4                Past exams
10                   Essays
5                     Other
9             Tutorial work
6                 Practical
1            Book solutions
Name: category_name, dtype: object

In [31]:
doc_views.query('category_name.isna() == False')[['category_id', 'category_name']].drop_duplicates().reset_index(drop=True).set_index('category_id')['category_name'].to_dict()

{3: 'Lecture notes',
 7: 'Summaries',
 8: 'Mandatory assignments',
 4: 'Past exams',
 10: 'Essays',
 5: 'Other',
 9: 'Tutorial work',
 6: 'Practical',
 1: 'Book solutions'}

In [32]:
category_dict = doc_views.query('category_name.isna() == False')[['category_id', 'category_name']].drop_duplicates().reset_index(drop=True).set_index('category_id')['category_name']

In [33]:
category_dict[9]

'Tutorial work'

In [34]:
def fill_nans(col_id): 
    return category_dict[col_id]

In [35]:
doc_views['category_name'] = doc_views['category_id'].apply(fill_nans)

## Анонимизация данных

In [36]:
doc_views.head()

,document_id,user_id,viewed_at,document_title,course_name,university_name,category_id,category_name,premium,rating,pages,word_count,id,name,last_name,passport_number
0,9738742,8469118,2020-11-30 22:59:56,Sosiale medier - 2. time,Sosiale medier,Handelshøyskolen BI,3,Lecture notes,0,0,1,151,1,Jacob,Edwards,1903155229
1,2042372,13198098,2020-11-30 22:59:52,Bedriften-kortsvarsspørsmål,Bedriften,Handelshøyskolen BI,7,Summaries,1,50,23,9551,2,Felicia,Smith,1466748063
2,3653898,11182794,2020-11-30 22:59:41,Sammendrag-i-markedsføring,Markedsføring-grunnkurs,Norges teknisk-naturvitenskapelige universitet,3,Lecture notes,0,1,107,37572,3,Erin,Parker,4864835140
3,7968789,13004670,2020-11-30 22:59:37,"Dydsetikk og abort, Hursthouse",Examen philosophicum,Universitetet i Oslo,8,Mandatory assignments,1,3,6,929,4,Morgan,Mcguire,1837181476
4,2036442,12525936,2020-11-30 22:59:30,Kortsvar-bedriften - Sammendrag Bedriften,Bedriften,Handelshøyskolen BI,7,Summaries,1,228,21,5567,5,Lisa,Erickson,5567324499


### Функция каширования данных

In [37]:
def hash_passport(passport): 
    return hashlib.sha256(passport.encode()).hexdigest()

#### Использование метода работы с колонкой добавления apply (берем исходную колонку находи значение и переписываем)

In [38]:
doc_views['passport_number'] = doc_views['passport_number'].apply(hash_passport)

### Убираем из колонки фамилию last_name

In [39]:
doc_views = doc_views.drop(columns='last_name')

## Обработка дублей

### Нахождение дублей

In [40]:
doc_views.duplicated().sum()

np.int64(0)

In [41]:
payments.duplicated().sum()

np.int64(48)

### Удаление дублей

In [42]:
payments = payments.drop_duplicates()

# 4 Загрузка данных в ChatGPT. Генерация гипотез. Исследовательский анализ данных

## Выгрузка файлов из Jupyter Notebook с функцией to_csv

In [43]:
doc_views.to_csv('doc_views_result.csv',index=False)

In [44]:
payments.to_csv('payments_result.csv',index=False)

# 5 Исследовательский анализ данных (EDA)

## Локация покупателей

In [45]:
payments['shopper_locale'].unique

<bound method Series.unique of 36      no
37      no
38      no
39      no
40      no
        ..
3365    no
3366    no
3367    no
3368    no
3369    no
Name: shopper_locale, Length: 3334, dtype: object>

## какие валюты использовались при платежах, используем функцию currency

In [46]:
payments.currency.unique()

array(['NOK'], dtype=object)

## Продолжая работать с датасетом Payments, сгруппируем его и применим к колонке локации функцию sum:


In [47]:
payments.groupby('shopper_locale')['amount'].sum()

shopper_locale
da             0
de             0
de_ch          0
en             0
en_au      47985
en_ca          0
en_gb          0
en_ie          0
en_nz          0
en_us          0
es         11997
hk             0
in             0
it             0
ja             0
nl             0
no       4654635
row            0
sg             0
sv             0
vn             0
Name: amount, dtype: int64

## Такой вид представления непривлекателен для заказчика, поэтому мы построим чарт с помощью такого кода:

In [48]:
plt.figure(figsize=(10, 6)) # Задаем размер графика
locale_amounts.plot(kind='bar', color='skyblue') # Создаем столбчатую диаграмму
plt.title('Сумма платежей по локациям покупателей') # Заголовок диаграммы
plt.xlabel('Локация покупателя') # Подпись оси Х
plt.ylabel('Сумма платежей') # Подпись оси Y
plt.xticks(rotation=45) # Поворот подписей на оси X для лучшей читаемости
plt.grid(True) # Включаем сетку для удобства восприятия
plt.show()

NameError: name 'locale_amounts' is not defined

<Figure size 1000x600 with 0 Axes>

In [49]:
locale_amounts = payments.groupby('shopper_locale')['amount'].sum()

## Влияние пробных периодов на последующие платежи

Эта гипотеза звучит достаточно обоснованно, поскольку на рынке часто встречаются компании, предлагающие пробные периоды для своих продуктов, и они хотят понимать, действительно ли пробный период способствует привлечению покупателей. Интуитивно кажется, что ответ положительный, поскольку пользователи, как правило, привыкают к продукту. Однако заказчики не доверяют интуитивным выводам, поэтому необходимо подтвердить (или опровергнуть) эту гипотезу с помощью данных.

### 1 Чтобы оставить в таблице Payments только те данные, в которых были пробные периоды (графа is_trial), используем уже знакомую функцию фильтрации. Прописываем:

In [50]:
payments.query('is_trial == 1')

,id,created_at,user_id,amount,brand_code,shopper_locale,currency,event_code,is_trial,trial_days
36,2663248,2020-11-30 21:06:53,13204266.0,0,visa,no,NOK,AUTHORISATION,1,30
38,2663236,2020-11-30 21:03:04,13204246.0,0,mc,no,NOK,AUTHORISATION,1,30
39,2663168,2020-11-30 20:59:30,13131658.0,0,visa,no,NOK,AUTHORISATION,1,30
40,2663144,2020-11-30 20:51:15,13203938.0,0,visa,no,NOK,AUTHORISATION,1,30
42,2663123,2020-11-30 20:44:42,13203780.0,0,visa,no,NOK,AUTHORISATION,1,30
...,...,...,...,...,...,...,...,...,...,...
3363,2494822,2020-11-01 11:27:04,12488536.0,0,visa,no,NOK,AUTHORISATION,1,30
3366,2494297,2020-11-01 08:33:23,12487016.0,0,mc,no,NOK,AUTHORISATION,1,30
3367,2494269,2020-11-01 08:05:53,12486818.0,0,visa,no,NOK,AUTHORISATION,1,30
3368,2494216,2020-11-01 07:37:35,12486620.0,0,mc,no,NOK,AUTHORISATION,1,30


### 2 Получив эти значения, запросим список пользователей (user_id), которые были на пробном периоде, оставив при этом флаг "is trial":

In [51]:
payments.query('is_trial == 1').drop_duplicates()[['user_id', 'is_trial']]

,user_id,is_trial
36,13204266.0,1
38,13204246.0,1
39,13131658.0,1
40,13203938.0,1
42,13203780.0,1
...,...,...
3363,12488536.0,1
3366,12487016.0,1
3367,12486818.0,1
3368,12486620.0,1


### 3. Упорядочиваем значения индекса уже знакомой командой reset_index, дописывая её в той же ячейке:

In [52]:
payments.query('is_trial == 1').drop_duplicates()[['user_id', 'is_trial']].reset_index(drop=True)

,user_id,is_trial
0,13204266.0,1
1,13204246.0,1
2,13131658.0,1
3,13203938.0,1
4,13203780.0,1
...,...,...
3072,12488536.0,1
3073,12487016.0,1
3074,12486818.0,1
3075,12486620.0,1


### 4. Сохраняем полученный результат в таблицу, дописывая в начало строки trial_users:

In [53]:
trial_users = payments.query('is_trial == 1').drop_duplicates()[['user_id', 'is_trial']]

### 5. Чтобы узнать пользователей, которые совершали покупки, в таблице Payments отфильтровываем строки, в которых значение amount больше 0:

In [54]:
payments.query('amount>0')

,id,created_at,user_id,amount,brand_code,shopper_locale,currency,event_code,is_trial,trial_days
37,2663241,2020-11-30 21:04:38,9007774.0,11997,visa,no,NOK,AUTHORISATION,0,0
41,2663124,2020-11-30 20:44:55,5780180.0,11997,visa,no,NOK,AUTHORISATION,0,0
62,2662559,2020-11-30 18:44:14,7763096.0,11997,paypal,no,NOK,AUTHORISATION,0,0
75,2662270,2020-11-30 17:50:11,13199820.0,11997,mc,no,NOK,AUTHORISATION,0,0
82,2662103,2020-11-30 17:08:15,8285304.0,11997,visa,no,NOK,AUTHORISATION,0,0
...,...,...,...,...,...,...,...,...,...,...
3319,2500024,2020-11-02 08:03:57,7532526.0,11997,mc,no,NOK,AUTHORISATION,0,0
3327,2498020,2020-11-01 22:27:34,12499356.0,11997,visa,no,NOK,AUTHORISATION,0,0
3355,2495807,2020-11-01 14:43:59,9816978.0,11997,visa,no,NOK,AUTHORISATION,0,0
3364,2494811,2020-11-01 11:19:18,12488456.0,11997,visa,no,NOK,AUTHORISATION,0,0


### 6. Избавляемся от дублей, оставив два поля: пользователя и продажу. Для этого добавляем к ранее прописанному коду функцию 

In [55]:
drop_duplicates: payments.query('amount>0').drop_duplicates()[['user_id', 'amount']]

### 7. Чтобы сохранить этот результат в переменную payed_users, добавляем в начало прописанной строки:



In [56]:
payed_users = payments.query('amount>0').drop_duplicates()[['user_id', 'amount']]

В результате мы получили две таблицы: пользователи с пробным периодом и пользователи, совершившие покупку.

Чтобы понять, сколько из купивших пользователей было на пробном периоде, нужно объединить эти таблицы: так мы сможем понять, что пользователь, купивший продукт, ранее был в списках пользователей, проходивших пробный период.

### 8. Для начала взглянем на обе эти таблицы. Прописываем для таблицы пробного периода:


In [57]:
trial_users.head()

,user_id,is_trial
36,13204266.0,1
38,13204246.0,1
39,13131658.0,1
40,13203938.0,1
42,13203780.0,1


### 9. То же самое прописываем для таблицы с покупателями:

In [58]:
payed_users.head()

,user_id,amount
37,9007774.0,11997
41,5780180.0,11997
62,7763096.0,11997
75,13199820.0,11997
82,8285304.0,11997


### 10. Чтобы объединить таблицы(в pandas используются методы merge и join. Здесь мы используем метод merge.), прописываем название таблицы с покупками и слева к ней присоединяем таблицу с данными о тех, кто был на пробном периоде:


In [59]:
joined_t = payed_users.merge(trial_users, how='left')

*Здесь важно не перепутать: если мы сделаем наоборот, то мы получим информацию о том, кто из тех, кто был на пробном периоде, в дальнейшем купил продукт. Это тоже ответит на вопрос, но немного на другой. Сейчас нам интересно узнать, кто из тех, кто купил продукт, был на пробном периоде*

### 11. Чтобы посмотреть, какая таблица получилась, прописываем уже знакомую функцию head:


In [60]:
joined_t.head()

,user_id,amount,is_trial
0,9007774.0,11997,NaN
1,5780180.0,11997,NaN
2,7763096.0,11997,NaN
3,13199820.0,11997,1.0
4,8285304.0,11997,NaN


12. Имея такую таблицу, мы можем посчитать долю тех, кто был на пробном периоде, от общего числа пользователей. Для этого просто из созданной таблицы просуммируем колонку is_trial:


In [61]:
joined_t['is_trial'].sum()

np.float64(19.0)

### 13. Теперь посчитаем, сколько всего в таблице клиентов (user_id), используя функцию count:


In [62]:
joined_t['user_id'].count()

np.int64(257)

Выполнив этот код, мы получим ответ: 257 человек.

Чтобы оценить долю тех, кто имел пробный период, из тех, кто оплатил, нужно поделить количество тех, кто пользовался пробным периодом, на общее количество пользователей.

Чтобы представить это в удобочитаемом виде, используем функцию print, которая может заключить в себя текст, а также значение round, которое округляет получаемый результат.

14. Прописываем:

In [63]:
print ('Доля тех, кто имел пробный период, из тех, кто оплатил',round(joined_t['is_trial'].sum()/joined_t['user_id'].count()*100,1),'%')

Доля тех, кто имел пробный период, из тех, кто оплатил 7.4 %


Мы узнали, что доля тех, кто прошёл пробный период и после этого купил продукт, составялет 7,4%.

Таким образом, мы провели аналитику и проверили гипотезу.